# Camera Calibration Assignment Q7

------

### Importing the required libs

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

### Utility Functions

In [2]:
def a_T(xi, Xi):
    '''
        Description : Calculates the a_xi.T and a_yi.T vectors
        
        params : xi - Image Coordinate (2,1)
                 Xi - World Coordinate (3,1)
        
        return : a_xi.T - (1, 12)
                 a_yi.T - (1, 12)
    '''
    xi_a_T = np.array([-Xi[0], -Xi[1], -Xi[2], -1, 0, 0, 0, 0, xi[0]*Xi[0], xi[0]*Xi[1], xi[0]*Xi[2], xi[0]])
    yi_a_T = np.array([0, 0, 0, 0, -Xi[0], -Xi[1], -Xi[2], -1, xi[1]*Xi[0], xi[1]*Xi[1], xi[1]*Xi[2], xi[1]])
    
    return xi_a_T, yi_a_T


def get_M(x, X):
    '''
        Description : Create the M matrix (ie xi_a_T & yi_a_T stacked, for each points)
        
        params : x - matrix contaiing all the image coordinates (I, 2)
                 X - matrix contaiing all the world coordinates (I, 3)
        
        return : M matrix (2I x 12) where I is the total number of points
    '''
    
    I = x.shape[0]
    M = np.zeros((2*I, 12))
    
    for i in range(I):
        xi_a_T, yi_a_T = a_T(x[i], X[i])
        M[2*i] = xi_a_T
        M[2*i+1] = yi_a_T
    
    return M

### Initializing the Coordinates

In [3]:
# World coordinates
X = np.array([[9,9,9],
              [9,7,9],
              [7,9,9],
              [7,7,9],
              [9,9,11],
              [9,7,11]])

# Image coordinates
x = np.array([[4,4],
              [4,2],
              [2,4],
              [2,2],
              [2,2],
              [2,1]])

### Creating M matrix

In [4]:
M = get_M(x, X)

print(M, M.shape)

[[ -9.  -9.  -9.  -1.   0.   0.   0.   0.  36.  36.  36.   4.]
 [  0.   0.   0.   0.  -9.  -9.  -9.  -1.  36.  36.  36.   4.]
 [ -9.  -7.  -9.  -1.   0.   0.   0.   0.  36.  28.  36.   4.]
 [  0.   0.   0.   0.  -9.  -7.  -9.  -1.  18.  14.  18.   2.]
 [ -7.  -9.  -9.  -1.   0.   0.   0.   0.  14.  18.  18.   2.]
 [  0.   0.   0.   0.  -7.  -9.  -9.  -1.  28.  36.  36.   4.]
 [ -7.  -7.  -9.  -1.   0.   0.   0.   0.  14.  14.  18.   2.]
 [  0.   0.   0.   0.  -7.  -7.  -9.  -1.  14.  14.  18.   2.]
 [ -9.  -9. -11.  -1.   0.   0.   0.   0.  18.  18.  22.   2.]
 [  0.   0.   0.   0.  -9.  -9. -11.  -1.  18.  18.  22.   2.]
 [ -9.  -7. -11.  -1.   0.   0.   0.   0.  18.  14.  22.   2.]
 [  0.   0.   0.   0.  -9.  -7. -11.  -1.   9.   7.  11.   1.]] (12, 12)


### Theory

```math

Now that we have M, we know that 

=> M . p should be 0 
=> M * p = 0 

This can be solved using SVD(Singular Value Decomposition)

=> M = U * S * V.T

And as all these matrices are arranged in a decreasing order of importance(contribution) 
to minimize the value of M . p and equate it to 0 we need to choose the least contributing 
value of V.T (ie the last row of V.T)

```

### Taking the SVD of M

In [5]:
U, S, VT = np.linalg.svd(M)

print(U.shape, S.shape, VT.shape)

(12, 12) (12,) (12, 12)


### Verifying the value of p

In [6]:
p = VT[-1]

print("p =\n", p,'\n\n')

print("M . p =\n", np.dot(M, p))

p =
 [ 0.12451456 -0.          0.         -0.62257281 -0.          0.12451456
  0.         -0.62257281 -0.         -0.          0.06225728 -0.43580096] 


M . p =
 [ 0.  0.  0. -0. -0.  0. -0. -0. -0. -0. -0. -0.]


As we can see that ``M . p = 0``

In [7]:
print("P =\n", p.reshape((3,4)))

P =
 [[ 0.12451456 -0.          0.         -0.62257281]
 [-0.          0.12451456  0.         -0.62257281]
 [-0.         -0.          0.06225728 -0.43580096]]


### Verifying the values of x and X using p

In [8]:
P = p.reshape((3,4))
X = np.append(X, np.ones((6,1)), axis=1)

In [9]:
expected_x = np.dot(P, X.T)

print(expected_x/expected_x[-1])

[[4. 4. 2. 2. 2. 2.]
 [4. 2. 4. 2. 2. 1.]
 [1. 1. 1. 1. 1. 1.]]


```

As we can see we have retrieved the image coordinates 'x' from P and X

```